In [3]:
%pip install matplotlib


     ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/8.1 MB 330.3 kB/s eta 0:00:25
     ---------------------------------------- 0.1/8.1 MB 525.1 kB/s eta 0:00:16
      --------------------------------------- 0.1/8.1 MB 819.2 kB/s eta 0:00:10
     - -------------------------------------- 0.2/8.1 MB 1.4 MB/s eta 0:00:06
     - -------------------------------------- 0.3/8.1 MB 1.5 MB/s eta 0:00:06
     - -------------------------------------- 0.4/8.1 MB 1.4 MB/s eta 0:00:06
     -- ------------------------------------- 0.5/8.1 MB 1.7 MB/s eta 0:00:05
     --- ------------------------------------ 0.7/8.1 MB 1.8 MB/s eta 0:00:05
     ---- ----------------------------------- 0.8/8.1 MB 2.0 MB/s eta 0:00:04
     ---- ----------------------------------- 0.9/8.1 MB 2.0 MB/s eta 0:00:04
     ----- ---------------------------------- 1.0/8.1 MB 2.1 MB/s eta 0:00:04
     ------ --------------------------------- 1.2/8.1 MB 2.2 MB/s


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%pip install keras-tuner
%pip install visualkeras

     ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
     --------- ----------------------------- 30.7/129.1 kB 1.3 MB/s eta 0:00:01
     ----------- ------------------------- 41.0/129.1 kB 388.9 kB/s eta 0:00:01
     -------------------------- ---------- 92.2/129.1 kB 871.5 kB/s eta 0:00:01
     ------------------------------------ 129.1/129.1 kB 843.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/44.9 kB ? eta -:--:--
     ---------------------------------- --- 41.0/44.9 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 44.9/44.9 kB 737.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
      --------------------------------------- 0.0/2.7 MB 2.0 MB/s eta 0:00:02
     -- ------------------------------------- 0.2/2.7 MB 2.2 MB/s eta 0:00:02
     ----- ---------------------------------- 0.4/2.7 MB 3.1 MB/s eta 0:00:01
     ------------ --------------------------- 0.8/2.7 MB 4.7 MB/s eta 0:00:01
     ------------------- -------------------- 1.3/2.7 MB 5.9 MB/s eta 0:00:01
     ----------------------------- ---------- 2.0/2.7 MB 7.5 MB/s eta 0:00:01
     ---------------------------------------  2.7/2.7 MB 9.0 MB/s eta 0:00:01
     ---------------------------------------- 2.7/2.7 MB 8.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install numpy
%pip install tensorflow
%pip install matplotlib
%pip install opencv-python
%pip install pandas
%pip install pillow
%pip install visualkeras
%pip install keras-tuner
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
     ---------------------------------------- 0.1/11.1 MB 1.1 MB/s eta 0:00:11
      --------------------------------------- 0.2/11.1 MB 1.5 MB/s eta 0:00:08
     - -------------------------------------- 0.5/11.1 MB 2.6 MB/s eta 0:00:05
     -- ------------------------------------- 0.7/11.1 MB 3.1 MB/s eta 0:00:04
     -- ------------------------------------- 0.8/11.1 MB 3.4 MB/s eta 0:00:03
     ---- ----------------------------------- 1.2/11.1 MB 3.7 MB/s eta 0:00:03
     ---- ----------------------------------- 1.3/11.1 MB 3.5 MB/s eta 0:00:03
     ----- ---------------------------------- 1.5/11.1 MB 3.9 MB/s eta 0:00:03
     ------ --------------------------------- 1.7/11.1 MB 3.9 MB/s eta 0:00:03
     ------- -------------------------------- 2.0/11.1 MB 4.0 MB/s eta 0:00:03
     -------- ------------------------------- 2.4/11.1 MB 4.4 MB/s


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os,glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import PIL
from tensorflow.keras.utils import load_img, img_to_array, array_to_img
from tensorflow.keras import layers, models, optimizers, metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import visualkeras as vk
import keras_tuner as kt
from keras_tuner import HyperParameters as hp
from sklearn.metrics import accuracy_score, confusion_matrix,recall_score,precision_score,classification_report,ConfusionMatrixDisplay
folder='Keras_tuner'
os.makedirs(folder, exist_ok=True)